# Processando e explorando dados em um banco relacional

## Importando pacotes

In [12]:
import pandas as pd
import numpy as np
import webbrowser

from nycflights13 import airports, flights, planes, airlines, weather
from fpdf import FPDF
from bokeh.io import export_png
from bokeh.plotting import output_notebook, show, figure
from bokeh.models import ColumnDataSource, HoverTool

## Preparando os dataframes

In [2]:
df_flights = flights.copy(deep=True)

### Atrasos

In [3]:
# formatando datas
df_flights['data'] = pd.to_datetime(df_flights[['day','month']].assign(Year=2013)).astype('string')

# somando os atrasos
df_flights['sum'] = df_flights[['dep_delay','arr_delay']].sum(axis=1,skipna=True)# skipna irá ignorar todos os valores null/nan

df_flights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,data,sum
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z,2013-01-01,13.0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z,2013-01-01,24.0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z,2013-01-01,35.0
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z,2013-01-01,-19.0
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z,2013-01-01,-31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,...,NaN,JFK,DCA,NaN,213,14,55,2013-09-30T18:00:00Z,2013-09-30,0.0
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,...,NaN,LGA,SYR,NaN,198,22,0,2013-10-01T02:00:00Z,2013-09-30,0.0
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,...,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30T16:00:00Z,2013-09-30,0.0
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,...,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30T15:00:00Z,2013-09-30,0.0


In [4]:
# cria um DF para cada cálculo
func = {'media':'mean','desvio':'std','assimetria':'skew'}
dfs = {}
for key in func:
    df = df_flights.groupby(['month','day','data']).agg({'sum':func[key]}).reset_index().rename(columns={"sum":key})
    dfs.update({key:df})

# concatena os DFs
df_atraso = pd.concat([dfs['media'],dfs['desvio']['desvio'],dfs['assimetria']['assimetria']],axis=1)
df_atraso

,month,day,data,media,desvio,assimetria
0,1,1,2013-01-01,23.979810,92.828987,9.330473
1,1,2,2013-01-02,26.232238,76.435534,4.098157
2,1,3,2013-01-03,16.513129,64.018518,3.628782
3,1,4,2013-01-04,6.974863,56.424330,3.568580
4,1,5,2013-01-05,4.188889,51.812762,5.835333
...,...,...,...,...,...,...
360,12,27,2013-12-27,10.777778,58.501226,2.825252
361,12,28,2013-12-28,4.716216,53.085805,4.151023
362,12,29,2013-12-29,40.173423,94.004181,2.762140
363,12,30,2013-12-30,20.445248,65.934058,3.486107


Infelizmente pelo método que utilizei não foi possível encontrar os valores de curtose para cada dia, conforme a célula abaixo.
Isso acontece devido a um problema na função "groupby", com [issue](https://github.com/pandas-dev/pandas/issues/40139) aberta ainda não solucionada.

In [5]:
curtose = df_flights.groupby(['month','day','data']).agg({'sum':'kurt'}).reset_index().rename(columns={"sum":'curtose'})
curtose

AttributeError: 'SeriesGroupBy' object has no attribute 'kurt'

### Plotagem

Para criar o gráfico, estou usando a biblioteca Bokeh, que permite fazer gráficos interativos baseado em JavaScript.

In [10]:
# construindo a base de dados
datas = np.array(df_atraso['data'], dtype=np.datetime64)
source = ColumnDataSource(data=dict(datas=datas,media=df_atraso['media'],desvio=df_atraso['desvio']))

# base do gráfico
plot_atraso = figure(
    title="Média e desvio-padrão dos atrasos de vôos da cidade de Nova Iorque em 2013",
    x_axis_type="datetime",
    height=600,
    width=1200,
)

# adiciona as linhas
l1 = plot_atraso.line("datas","media",source=source,legend_label="Média",color="red",line_width=2)
l2 = plot_atraso.line("datas","desvio",source=source,legend_label="Desvio-padrão",color="blue",line_width=2)

# tooltips
tt1="media: @media"
tt2="desvio: @desvio"

# adiciona mousehover
h1 = HoverTool(renderers=[l1],tooltips=tt1,mode="mouse")
h2 = HoverTool(renderers=[l2],tooltips=tt2,mode="mouse")
plot_atraso.add_tools(h1,h2)

# resultado
output_notebook()
show(plot_atraso)

Loading BokehJS ...

In [14]:
export_png(plot_atraso,filename="plot.png")

RuntimeError: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.

In [ ]:
airports

In [ ]:
planes

In [ ]:
airlines

In [ ]:
weather